In [7]:
# pose sequence as a NLI premise and label as a hypothesis
from transformers import AutoModelForSequenceClassification, AutoTokenizer
nli_model = AutoModelForSequenceClassification.from_pretrained('facebook/bart-large-mnli')
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large-mnli')

In [23]:
import torch

In [70]:

sequence = "one day I will see the world"
batch_sentences = [
    "one day I will see the world",
    "Don't think he knows about second breakfast, Pip.",
    "What about elevensies?"
]
labels = ['travel', 'cooking', 'dancing']
premise = sequence
hypothesis = f'This example is {labels[0]}'
hypotheses = [f'This example is {label}.' for label in labels]

In [59]:
tokens = tokenizer(batch_sentences, padding=True, return_tensors="pt") # truncation=True
ids = tokenizer.convert_tokens_to_ids(tokens)
input_ids = torch.tensor(ids)
print(tokens)
print("Input IDs:", input_ids)

output = nli_model(**tokens)
print("Logits:", output.logits)

{'input_ids': tensor([[    0,  1264,   183,    38,    40,   192,     5,   232,     2,     1,
             1,     1,     1],
        [    0,  6766,    75,   206,    37,  2215,    59,   200,  7080,     6,
         30533,     4,     2],
        [    0,  2264,    59, 19353,    29,   918,   116,     2,     1,     1,
             1,     1,     1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0]])}
Input IDs: tensor([3, 3])
Logits: tensor([[-2.6491,  3.7369, -1.7229],
        [ 0.1108,  2.3191, -2.5804],
        [ 0.6952, -0.1324, -0.9485]], grad_fn=<AddmmBackward0>)


In [66]:
pairs = [(seq, hypo) for seq in batch_sentences for hypo in hypotheses]
x = tokenizer.batch_encode_plus(pairs, return_tensors="pt", padding=True)

output = nli_model(**tokens)
print("Logits:", output.logits)

Logits: tensor([[-2.6491,  3.7369, -1.7229],
        [ 0.1108,  2.3191, -2.5804],
        [ 0.6952, -0.1324, -0.9485]], grad_fn=<AddmmBackward0>)


In [ ]:

# run through model pre-trained on MNLI
x = tokenizer.encode(premise, hypothesis, return_tensors='pt',
                     truncation_strategy='only_first')
logits = nli_model(x)

# we throw away "neutral" (dim 1) and take the probability of
# "entailment" (2) as the probability of the label being true 
entail_contradiction_logits = logits[:,[0,2]]
probs = entail_contradiction_logits.softmax(dim=1)
prob_label_is_true = probs[:,1]

In [11]:
x

tensor([[   0, 1264,  183,   38,   40,  192,    5,  232,    2,    2,  713, 1246,
           16, 1504,    4,    2]])